In [4]:
import pandas as pd
import numpy as np
# load the source data for each panel 
panel_a = pd.read_excel('./fig2_sourcedata.xlsx',engine='openpyxl',sheet_name='Panel_a',index_col=0)
panel_b = pd.read_excel('./fig2_sourcedata.xlsx',engine='openpyxl',sheet_name='Panel_b',index_col=0)
panel_c = pd.read_excel('./fig2_sourcedata.xlsx',engine='openpyxl',sheet_name='Panel_c',index_col=0)
panel_d = pd.read_excel('./fig2_sourcedata.xlsx',engine='openpyxl',sheet_name='Panel_d',index_col=0)
panel_e = pd.read_excel('./fig2_sourcedata.xlsx',engine='openpyxl',sheet_name='Panel_e',index_col=0)
panel_f = pd.read_excel('./fig2_sourcedata.xlsx',engine='openpyxl',sheet_name='Panel_f',index_col=0)
panel_g = pd.read_excel('./fig2_sourcedata.xlsx',engine='openpyxl',sheet_name='Panel_g',index_col=0)

In [5]:
def bootstrap(arr,itr=20000):
    boots = []
    for i in range(itr):
        boots.append(np.median(np.random.choice(arr,arr.shape,replace=True)))
        
    return(boots)

def print_bootstrap_results(df,column,itr=20000):
    straps = np.array(bootstrap(df[column] ))
    median_value = np.round(np.median(df[column]),2)
    low_bound = np.round(np.percentile(straps,0.5),2)
    up_bound = np.round(np.percentile(straps,99.5),2)
    print('Results for: ',column)
    print('Median: ', median_value)
    print('99% CI: [', low_bound, ', ', up_bound, ']')
    print('')

print_bootstrap_results(panel_a,'both')
print_bootstrap_results(panel_a,'both_masked')
print_bootstrap_results(panel_a,'eng_masked')
print_bootstrap_results(panel_a,'spa_masked')



Results for:  both
Median:  47.24
99% CI: [ 45.83 ,  48.23 ]

Results for:  both_masked
Median:  58.1
99% CI: [ 56.93 ,  59.27 ]

Results for:  eng_masked
Median:  52.9
99% CI: [ 51.57 ,  55.57 ]

Results for:  spa_masked
Median:  62.95
99% CI: [ 61.34 ,  64.95 ]



In [7]:
# get relevant stats 
from gimutil.result_analysis import comparisons
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from scipy.stats import shapiro
t,p = comparisons.correlation_permutation(panel_d['English LFS'],panel_d['Spanish LFS'],corr=spearmanr,n_permute=20000)
print('LFS')
print(t)
print(p)
# print('HGA')
t,p = comparisons.correlation_permutation(panel_d['English HGA'],panel_d['Spanish HGA'],corr=spearmanr,n_permute=20000)
print('HGA')
print(t)
print(p)






LFS
0.9182441402673501
0.0
HGA
0.8470117652444606
0.0


In [11]:
from scipy.stats import mannwhitneyu
panel_b['before_1250'] = panel_b.Day <= 1345
mannwhitneyu(panel_b['Accuracy'][(panel_b.Language == 'Overall') & (panel_b.before_1250 == True)].values,
        panel_b['Accuracy'][(panel_b.Language == 'Overall') & (panel_b.before_1250 == False)].values)


MannwhitneyuResult(statistic=9.0, pvalue=0.5476190476190477)

In [12]:
from scipy.stats import wilcoxon

cats = ['MCD','Semantic\nsimilarity','Same\nlanguage']
ps = []
tests = []
for i,cat in enumerate(cats):
    for cat2 in cats[(i+1):]:
        tests.append((cat,cat2))
        ps.append(wilcoxon(panel_g['Variance explained'][panel_g.Var == cat],
                               panel_g['Variance explained'][panel_g.Var == cat2]).pvalue)
        print((cat,cat2))
        print(ps[-1])

from statsmodels.stats.multitest import multipletests
multipletests(ps,alpha=0.01,method='holm')[1]

('MCD', 'Semantic\nsimilarity')
0.0
('MCD', 'Same\nlanguage')
0.0
('Semantic\nsimilarity', 'Same\nlanguage')
0.0


array([0., 0., 0.])